In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
from data_utils import city_data_utils, weather_utils, airport_data_utils

In [ ]:
load_dotenv()

# Access your SQL password
SQL_PW = os.getenv("SQL_PW")

#### Basic Details of Cities

In [ ]:
cities = ['Berlin', 'Hamburg', 'Munich']
cities_geodata_df = city_data_utils.scrape_city_data(cities)
cities_geodata_df

In [ ]:
# Create connection string to connect to MySQL database
def create_connection_string():
  schema = "gans_weather"
  host = "127.0.0.1"
  user = "root"
  password = SQL_PW
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

Send `cities_geodata_df` to the `cities` table in the Database

In [ ]:
cities_geodata_df.to_sql('cities',
                    if_exists='append',
                    con=create_connection_string(),
                    index=False)

Create and send population data using the `cities` table

In [ ]:
def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

In [ ]:
cities_population_df = city_data_utils.scrape_population_data(fetch_cities_data(create_connection_string()))
cities_population_df

In [ ]:
cities_population_df.to_sql('population',
                    if_exists='append',
                    con=create_connection_string(),
                    index=False)

#### Get Weather Data of the Cities

In [ ]:


def store_weather_data(weather_df, connection_string):
  weather_df.to_sql('weather',
                    if_exists='append',
                    con=connection_string,
                    index=False)

  
def retrieve_and_send_data():
  connection_string = create_connection_string()
  cities_df = fetch_cities_data(connection_string)
  weather_df = weather_utils.fetch_weather_data(cities_df)
  store_weather_data(weather_df, connection_string)
  return "Data has been updated"

Send weather data to the database on MySQL

In [ ]:
retrieve_and_send_data()

#### Airports Data

In [ ]:
cities_df = fetch_cities_data(create_connection_string())
cities_df

In [ ]:
airports_df = airport_data_utils.get_airports(cities_df)
airports_df

In [ ]:
airports_df_cl = airports_df.loc[airports_df['iata'] != 'TXL', ['city_id', 'icao', 'name']]
airports_df_cl = airports_df_cl.rename(columns= {'icao':'icao_code', 'name': 'airport_name'})
airports_df_cl

Send airports data to the database on MySQL

In [ ]:
airports_df_cl.to_sql('cities_airports',
                      if_exists='append',
                    con=create_connection_string(),
                    index=False)

#### Flights Data

In [ ]:
flights_df =  airport_data_utils.fetch_airports_data(airports_df_cl)
flights_df

Send flights data to the database on MySQL

In [ ]:
flights_df.to_sql('flights',
                    if_exists='append',
                    con=create_connection_string(),
                    index=False)